In [ ]:
#Centralityのcsvファイルを読み込み、分析範囲ごとにRANKを計算・付与してcsvファイルを指定したフォルダに保存

import os
import glob
import pandas as pd

# 入力フォルダと出力フォルダを定義
input_folder = "output/Centrality"   # ←ここに対象フォルダのパスを指定
output_folder = "output/Centrality/ranked"  # 出力フォルダ

# 出力フォルダが存在しない場合は作成
os.makedirs(output_folder, exist_ok=True)

# 対象のスコア列
score_columns = [
    "Degree Centrality",
    "Betweenness Centrality",
    "Closeness Centrality",
    "Eigenvector Centrality"
]

# 入力フォルダ内のすべてのCSVファイルを処理
for file_path in glob.glob(os.path.join(input_folder, "*.csv")):

    #処理中のファイルを表示
    print(file_path)

    df = pd.read_csv(file_path)
    
    # 分析範囲ごとに順位を付けて新しい列を追加（NaNも考慮）
    for col in score_columns:
        # 念のため、数値変換（非数値はNaN化される）
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
        # 順位列を追加（NaNは自動的に除外される）
        df[f"{col}_Rank"] = df.groupby("分析範囲")[col].rank(method="min", ascending=False)
        
        # 整数として表示する場合、NaN以外をint化（見やすさのため）
        df[f"{col}_Rank"] = df[f"{col}_Rank"].apply(lambda x: int(x) if pd.notnull(x) else None)


    
    # 元ファイル名に '_ranked' を付けて出力ファイル名を作成
    base_name = os.path.basename(file_path)
    name_without_ext, ext = os.path.splitext(base_name)
    ranked_filename = f"{name_without_ext}_ranked{ext}"
    output_path = os.path.join(output_folder, ranked_filename)
    
    # BOM付きUTF-8で保存
    df.to_csv(output_path, index=False, encoding="utf-8-sig")
print("すべてのファイルを処理しました。")
